## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [24]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [26]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

print(len(y_train), y_train.shape, y_train[0])
print(len(y_test), y_test.shape, y_test[0])

50000 (50000, 10) [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
10000 (10000, 10) [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


C:\Users\d211\Anaconda3\envs\pyton_3.7\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [28]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3), padding='same',activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2),padding='same'))

#卷積組合
classifier.add(Convolution2D(32,(5,5),input_shape=(32,32,3),activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.25))

classifier.add(Convolution2D(32,(5,5),input_shape=(32,32,3),activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.25))

classifier.add(Convolution2D(32,(5,5),input_shape=(32,32,3),activation='relu', padding='same'))
classifier.add(Dropout(0.5))
classifier.add(BatchNormalization())

classifier.add(Convolution2D(64,(5,5),input_shape=(32,32,3),activation='relu', padding='same'))
classifier.add(BatchNormalization())



#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100)) #output_dim=100,activation=relu
classifier.add(Dense(100)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\Users\d211\Anaconda3\envs\pyton_3.7\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 263s 5ms/step - loss: 1.6169 - acc: 0.4668
Epoch 2/100
50000/50000 [==============================] - 254s 5ms/step - loss: 1.1294 - acc: 0.5990
Epoch 3/100
50000/50000 [==============================] - 254s 5ms/step - loss: 0.9576 - acc: 0.6658
Epoch 4/100
50000/50000 [==============================] - 256s 5ms/step - loss: 0.8478 - acc: 0.7020
Epoch 5/100
50000/50000 [==============================] - 254s 5ms/step - loss: 0.7661 - acc: 0.7310
Epoch 6/100
50000/50000 [==============================] - 255s 5ms/step - loss: 0.6978 - acc: 0.7562
Epoch 7/100
50000/50000 [==============================] - 258s 5ms/step - loss: 0.6530 - acc: 0.7708
Epoch 8/100
50000/50000 [==============================] - 256s 5ms/step - loss: 0.6123 - acc: 0.7854
Epoch 9/100
50000/50000 [==============================] - 255s 5ms/step - loss: 0.5793 - acc: 0.7969
Epoch 10/100
25700/50000 [==============>...............] - ETA: 2:05 - loss: 0.53

KeyboardInterrupt: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [23]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.16098002e-01, 1.59126193e-05, 1.17026396e-01, 2.07555884e-09,
        1.23824611e-01, 3.41737141e-05, 1.17292593e-03, 9.42336321e-02,
        7.41764680e-02, 7.16183684e-04, 5.76176420e-02, 7.60612893e-05,
        8.26359466e-02, 2.40595659e-06, 1.16536617e-01, 3.72799285e-08,
        1.07098721e-01, 2.18690673e-07, 1.08734041e-01, 1.08125797e-09]],
      dtype=float32)